In [1]:
import pandas as pd

df_application = pd.read_csv('application_record.csv')

df_application

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0
438553,6840222,F,N,N,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0
438554,6841878,F,N,N,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0
438555,6842765,F,N,Y,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,NaN,2.0


In [2]:
df_credit = pd.read_csv('credit_record.csv')

df_credit

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


In [3]:
df_credit['MONTHS_BALANCE'].unique()

array([  0,  -1,  -2,  -3,  -4,  -5,  -6,  -7,  -8,  -9, -10, -11, -12,
       -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25,
       -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38,
       -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51,
       -52, -53, -54, -55, -56, -57, -58, -59, -60])

In [4]:
df_credit['STATUS'].unique()

array(['X', '0', 'C', '1', '2', '3', '4', '5'], dtype=object)

In [13]:
df_credit1 = pd.DataFrame(df_credit.groupby(['ID'])['MONTHS_BALANCE'].agg(min))
df_credit1

,MONTHS_BALANCE
ID,
5001711,-3
5001712,-18
5001713,-21
5001714,-14
5001715,-59
...,...
5150482,-28
5150483,-17
5150484,-12


In [21]:
df_credit['STATUS'] = pd.to_numeric(df_credit['STATUS'], errors='coerce')

def overdue(STATUS):
    # Si el estado es 2 o superior (60 días o más de retraso), retorna 'Yes', de lo contrario, retorna 'No'
    return 'Yes' if STATUS >= 2 else 'No'

# Aplica la función a la columna STATUS para crear una nueva columna llamada 'ARIESGADO'
df_credit['Overdue'] = df_credit['STATUS'].apply(overdue)

df_credit


,ID,MONTHS_BALANCE,STATUS,value,Overdue
0,5001711,0,NaN,None,No
1,5001711,-1,0.0,None,No
2,5001711,-2,0.0,None,No
3,5001711,-3,0.0,None,No
4,5001712,0,NaN,None,No
...,...,...,...,...,...
1048570,5150487,-25,NaN,None,No
1048571,5150487,-26,NaN,None,No
1048572,5150487,-27,NaN,None,No
1048573,5150487,-28,NaN,None,No


In [28]:
def es_arriesgado(status):
    return 'Yes' if status in ['2', '3', '4', '5'] else 'No'

# Crear la columna 'dep_value' utilizando apply y una función lambda
df_credit['dep_value'] = df_credit['STATUS'].apply(lambda x: es_arriesgado(x))

# Agrupar y contar los valores 'Yes' para cada cliente
count_dep_values = df_credit.groupby('ID')['dep_value'].transform(lambda x: (x == 'Yes').sum())

# Asignar 'Yes' a los clientes con al menos un valor 'Yes', 'No' en caso contrario
df_credit['dep_value'] = 'No'
df_credit.loc[count_dep_values > 0, 'dep_value'] = 'Yes'

# Eliminar las filas duplicadas en df_credit
df_credit = df_credit.drop_duplicates(subset=['ID'])

# Mostrar el DataFrame resultante
df_credit

,ID,MONTHS_BALANCE,STATUS,value,Overdue,dep_value
0,5001711,0,NaN,None,NaN,No
4,5001712,0,NaN,None,NaN,No
23,5001713,0,NaN,None,NaN,No
45,5001714,0,NaN,None,NaN,No
60,5001715,0,NaN,None,NaN,No
...,...,...,...,...,...,...
1048494,5150482,-11,NaN,None,NaN,No
1048512,5150483,0,NaN,None,NaN,No
1048530,5150484,0,NaN,None,NaN,No
1048543,5150485,0,NaN,None,NaN,No


In [3]:
starter_month=pd.DataFrame(df_credit.groupby(['ID'])['MONTHS_BALANCE'].agg(min))
starter_month=starter_month.rename(columns={'MONTHS_BALANCE':'Account day'})
df_application=pd.merge(df_application,starter_month,how='left',on='ID')

df_credit['dep_value'] = None
df_credit['dep_value'][df_credit['STATUS'] =='2']='Yes'
df_credit['dep_value'][df_credit['STATUS'] =='3']='Yes'
df_credit['dep_value'][df_credit['STATUS'] =='4']='Yes'
df_credit['dep_value'][df_credit['STATUS'] =='5']='Yes'

cpunt=df_credit.groupby('ID').count()
cpunt['dep_value'][cpunt['dep_value'] > 0]='Yes'
cpunt['dep_value'][cpunt['dep_value'] == 0]='No'
cpunt = cpunt[['dep_value']]

df_application = pd.merge(df_application,cpunt,how='inner',on='ID')
df_application['Risky']=df_application['dep_value']
df_application.loc[df_application['Risky']=='Yes','Risky']=1
df_application.loc[df_application['Risky']=='No','Risky']=0
df_application.drop('dep_value',axis=1,inplace=True)

/tmp/ipykernel_26133/211919727.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_credit['dep_value'][df_credit['STATUS'] =='2']='Yes'
/tmp/ipykernel_26133/211919727.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_credit['dep_value'][df_credit['STATUS'] =='3']='Yes'
/tmp/ipykernel_26133/211919727.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_credit['dep_value'][df_credit['STATUS'] =='4']='Yes'
/tmp/ipykernel_26133/2

In [4]:
df_application

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,Account day,Risky
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-15.0,0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-14.0,0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-29.0,0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-4.0,0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-26.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,5149828,M,Y,Y,0,315000.0,Working,Secondary / secondary special,Married,House / apartment,-17348,-2420,1,0,0,0,Managers,2.0,-11.0,1
36453,5149834,F,N,Y,0,157500.0,Commercial associate,Higher education,Married,House / apartment,-12387,-1325,1,0,1,1,Medicine staff,2.0,-23.0,1
36454,5149838,F,N,Y,0,157500.0,Pensioner,Higher education,Married,House / apartment,-12387,-1325,1,0,1,1,Medicine staff,2.0,-32.0,1
36455,5150049,F,N,Y,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,-17958,-655,1,0,0,0,Sales staff,2.0,-9.0,1


In [5]:
from sklearn.model_selection import train_test_split
# split the data into train and test dataset
def data_split(df, test_size):
    train_df, test_df = train_test_split(df, test_size=test_size, random_state=42)
    # reset the indexes
    return train_df.reset_index(drop=True), test_df.reset_index(drop=True)


# we set the test_size to 0.2, which means that the train_size will be 0.8
df_app_train_original, df_app_test_original = data_split(df_application, 0.2)

ModuleNotFoundError: No module named 'sklearn'